In [1]:
import pickle

import keras

Using TensorFlow backend.


In [2]:
def limit_sentences(txt_str,num_words):
    
    return txt_str.split(' ')[:num_words]

In [4]:
query_response = pickle.load(open("../data/query_response_direct.p","rb"))